In [ ]:
import os
import pandas as pd
import re

# 1. 경로 설정
input_folder = '../2.혼잡도/4_same_weekend_data'
output_folder = '../2.혼잡도/5_same_hour_data'
os.makedirs(output_folder, exist_ok=True)

# 2. 파일 반복 처리
for filename in os.listdir(input_folder):
    if not filename.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, filename)
    print(f"📂 처리 중: {filename}")

    try:
        df = pd.read_csv(file_path, encoding="euc-kr")
    except Exception as e:
        print(f"❌ 파일 읽기 실패: {e}")
        continue

    # 3. 시간 컬럼 추출 (예: '05시30분', '06시00분' 등)
    time_cols = [col for col in df.columns if re.match(r"\d{2}시\d{2}분", col)]

    # 4. 시 단위 그룹핑: '05시30분' -> '05:00'으로 매핑
    hour_groups = {}
    for col in time_cols:
        hour_str = col[:2]  # '05' 추출
        hour_key = f"{hour_str}:00"  # '05:00' 포맷
        hour_groups.setdefault(hour_key, []).append(col)

    # 5. 시 단위 평균 계산 후 새로운 컬럼으로 추가
    for hour_key, cols in hour_groups.items():
        df[hour_key] = df[cols].mean(axis=1)

    # 6. 기존 시간 컬럼 삭제
    df.drop(columns=time_cols, inplace=True)

    # 7. 기본 정보 컬럼 리스트
    base_cols = ["평일주말", "호선", "역번호", "역명", "구분"]

    # 8. 시간 컬럼 정렬 (시간 순서대로)
    hour_cols = sorted(hour_groups.keys(), key=lambda x: int(x.split(':')[0]))

    # 9. 연번 컬럼 추가 (이미 있으면 삭제 후 재생성)
    if '연번' in df.columns:
        df.drop(columns=['연번'], inplace=True)
    df.insert(0, '연번', range(1, len(df) + 1))

    # 10. 최종 컬럼 순서 지정
    final_cols = ['연번'] + base_cols + hour_cols
    df = df[final_cols]

    # 11. 저장
    output_path = os.path.join(output_folder, f"hourlyavg_{filename}")
    try:
        df.to_csv(output_path, index=False, encoding="euc-kr")
        print(f"✅ 저장 완료: {output_path}\n")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
